In [1]:
import  os
import psutil


In [ ]:
def get_folder_size(path:str) -> int:
    totalSize : int= 0

    for dirpath,_,filenames in os.walk(path,onerror=lambda e:None):
        for f in filenames:
            try:
                fp = os.path.join(dirpath, f)   # full path of file
                if os.path.isfile(fp):
                    totalSize +=os.path.getsize(fp)

            except(PermissionError,FileNotFoundError):
                continue
    return totalSize

get_folder_size('F:\\')



129657716126

In [ ]:
def get_bigger_folder(root_Path  : str)  -> int:
    size :int = 0
    biggestSize : int = 0
    biggestFolder : str = None

    for entery in os.scandir(root_Path): # iterate over the entires
        print(entery)
        try:
    
            if entery.is_dir(follow_symlinks=False):
                size = get_folder_size(entery.path)
                if size> biggestSize:
                    biggestSize = size
                    biggestFolder = entery.path 
        except(PermissionError,FileExistsError,FileNotFoundError):
            continue
    return biggestFolder,biggestSize

get_bigger_folder('F:\\')


<DirEntry '$RECYCLE.BIN'>
<DirEntry 'aya'>
<DirEntry 'mine'>
<DirEntry 'msqlsrvr'>
<DirEntry 'System Volume Information'>


('F:\\mine', 123121355577)

In [12]:
import os
import psutil
from typing import Iterator, Tuple, List, Optional
import cProfile
import pstats
import io


def get_folder_size(path: str) -> int:
    """Return total size of a folder (in bytes)."""
    total_size: int = 0
    walker: Iterator[Tuple[str, List[str], List[str]]] = os.walk(path, onerror=lambda e: None)

    for dirpath, _, filenames in walker:
        for f in filenames:
            try:
                fp: str = os.path.join(dirpath, f)
                if os.path.isfile(fp):
                    total_size += os.path.getsize(fp)
            except (PermissionError, FileNotFoundError):
                continue
    return total_size


def bytes_to_gb(size_in_bytes: int) -> float:
    """Convert bytes to GB (2 decimal places)."""
    return round(size_in_bytes / (1024**3), 2)


def get_biggest_folder(root_path: str) -> Tuple[Optional[str], int]:
    """Return the biggest top-level folder in a partition."""
    biggest_folder: Optional[str] = None
    biggest_size: int = 0

    for entry in os.scandir(root_path):
        if entry.is_dir(follow_symlinks=False):
            try:
                size = get_folder_size(entry.path)
                if size > biggest_size:
                    biggest_size = size
                    biggest_folder = entry.path
            except (PermissionError, FileNotFoundError):
                continue
    return biggest_folder, biggest_size


def main():
    # Show available partitions first
    partitions = psutil.disk_partitions()
    print("Available partitions:")
    for p in partitions:
        print(f"- {p.device} ({p.mountpoint})")

    # Ask user for a partition
    user_choice = input("\nEnter the partition letter (e.g., C, D, E, F): ").strip().upper()
    partition_path = f"{user_choice}:\\"

    print(f"\n🔹 Partition: {partition_path}")
    try:
        # total partition file size
        total_size = get_folder_size(partition_path)
        print(f"   Total file size: {bytes_to_gb(total_size)} GB")

        # biggest top-level folder
        biggest_folder, biggest_size = get_biggest_folder(partition_path)
        if biggest_folder:
            print(f"   Biggest folder: {biggest_folder} — {bytes_to_gb(biggest_size)} GB")
        else:
            print("   No accessible folders found.")
    except PermissionError:
        print("   Skipped (Permission Denied)")
    except Exception as e:
        print(f"   Error: {e}")


if __name__ == "__main__":

    pr = cProfile.Profile()
    pr.enable()
    main()


    pr.disable()
    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats("cumulative")  # Sort by total time
    ps.print_stats(10)  # Show top 10 functions
    print("\n--- Profiling Report ---")
    print(s.getvalue())



Available partitions:
- C:\ (C:\)
- D:\ (D:\)
- E:\ (E:\)
- F:\ (F:\)

🔹 Partition: D:\
   Total file size: 176.82 GB
   Biggest folder: D:\تعليم — 80.07 GB

--- Profiling Report ---
         8498276 function calls (8498159 primitive calls) in 70.758 seconds

   Ordered by: cumulative time
   List reduced from 330 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       27    0.359    0.013   51.812    1.919 e:\installatinos\anaconda\Lib\asyncio\windows_events.py:443(select)
       27    0.573    0.021   27.802    1.030 e:\installatinos\anaconda\Lib\asyncio\windows_events.py:761(_poll)
   408369   27.437    0.000   27.437    0.000 {built-in method nt._path_isfile}
       22    0.578    0.026   24.283    1.104 C:\Users\ayama\AppData\Local\Temp\ipykernel_10336\3216605091.py:9(get_folder_size)
   408369    0.586    0.000   19.628    0.000 <frozen genericpath>:60(getsize)
   408369   19.042    0.000   19.042    0.000 {built-in method nt.